![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Carl_Friedrich_Gauss.jpg/330px-Carl_Friedrich_Gauss.jpg)

# Описание метода

Даны матрица $A$ размером $N\times N$, вектор $B$ высотой $N$. Требуется найти вектор $X$ такой, что $AX=B$.

Без терминологии линейной алгебры это записывается, как:

$
\begin{array}{lcr}
a_{11}x_1+\ldots+a_{1N}x_N &=& b_1 \\
\ldots & & \\
a_{N1}x_1+\ldots+a_{NN}x_N &=& b_N \\
\end{array}
$

## Прямой ход

Из второго уравнения вычитаем первое, умноженное на $\frac{a_{21}}{a_{11}}$. Из третьего вычитаем первое, умноженное на $\frac{a_{31}}{a_{11}}$. И т.д. до $N$-го. Получаем систему уравнений следующего вида.

$
\begin{array}{lcr}
a_{11}&x_1 +& a_{12} x_2 +\ldots+a_{1N}x_N &=& b_1 \\
0 &x_1 +& a'_{22} x_2 +\ldots+a'_{1N}x_N &=& b'_1 \\
& \ldots & & \\
0 &x_1 +& a'_{N2} x_2 +\ldots+a'_{NN}x_N &=& b'_N \\
\end{array}
$

Затем из третьего уравнения вычитаем второе, умноженное на $\frac{a_{32}}{a_{22}}$.

И т.д., пока не выполним такое же действие над последней строкой.

В итоге мы получим систему уравнений следующего вида.

$
\begin{array}{lcr}
a_{11}& x_1 +& a_{12}   & x_2 + & a''_{13} & x_3 + \ldots + a_{1N}   x_N &=& b_1 \\
0     & x_1 +& a''_{22} & x_2 + & a''_{23} & x_3 + \ldots + a''_{2N} x_N &=& b''_1 \\
0     & x_1 +& 0        & x_2 + & a''_{33} & x_3  +\ldots + a''_{3N} x_N &=& b''_3 \\
0     & x_1 +& 0        & x_2 + & 0        & x_3 + \ldots + a''_{4N} x_N &=& b''_4 \\
& & & \ldots & & & & \\
0 & x_1 +& & \ldots & +                  & 0 x_{N-1} + a''_{NN} x_N &=& b''_N \\
\end{array}
$


## Обратный ход

Очевидно, $x_N = \frac{b''_N}{a''_{NN}}$.

Далее, $x_i = \frac{b''_i - \sum_{j=i+1}^{N} a''_{ij}x_j}{a_{ii}}$

# Прямой и обратный ход в вектроной форме

Интересно? Тогда см. ниже. Неинтересно? Тоже см. ниже.

# Задание

Предлагается реализовать слкдующие недостающие функции

In [20]:
import numpy
from numpy import array
from numpy.linalg import norm
from numpy.linalg import solve as solve_out_of_the_box

def gauss(a, b):
    a = a.copy()
    b = b.copy()

    def forward():
        n = len(a)
        c = numpy.zeros((n, n+1))

        for i in range(n):
            for j in range(n):
                c[i,j]=a[i,j]
                c[i,j+1]=b[i]

        for k in range(0,n):
            if c[k,k] == 0:
                checker = -1
                for m in range(k, n):
                    if c[m, k] != 0:
                        checker = m
                if checker == -1:
                    k += 1
                else:
                    tmp = 0
                    for d in range(k,n+1):
                        tmp = c[k,d]
                        c[k,d] = c[checker,d]
                        c[checker,d] = tmp
                    for i in range(k+1,n):
                        tumpa = c[i,k]/c[k,k]
                        for j in range(k,n+1):
                            c[i,j] -= (c[k,j]*tumpa)
            else:
                for i in range(k+1,n):
                    tumpa = c[i,k]/c[k,k]
                    for j in range(k,n+1):
                        c[i,j] -= (c[k,j]*tumpa)

            for i in range(n):
                for j in range(n):
                    a[i,j]=c[i,j]
                    b[i]=c[i,j+1]



    def backward():
        n = len(a)
        x = numpy.zeros(len(b), dtype=float)
        x[n-1] = b[n-1]/a[n-1,n-1]

        for i in reversed(range(n-1)):
            sum = 0
            for j in range(i+1,n):
                sum += a[i,j]*x[j]
            x[i] = (b[i]-sum)/a[i,i]
        return x

    forward()
    x = backward()
    return x

a = array([
    [1.5, 2.0, 1.5, 2.0],
    [3.0, 2.0, 4.0, 1.0],
    [1.0, 6.0, 0.0, 4  ],
    [2.0, 1.0, 4.0, 3  ]
], dtype=float)

b = array([5, 6, 7, 8], dtype=float)

oob_solution = solve_out_of_the_box(a, b)
solution = gauss(a, b)

print(solution)
print("Макс отклонение компоненты решения:", norm(solution-oob_solution, ord=1))


[0.8490566  0.05660377 0.47169811 1.45283019]
Макс отклонение компоненты решения: 3.3584246494910985e-15
